# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../WeatherPy/weather_df.csv"
weather_dataframe = pd.read_csv(weather_csv_file)
#weather_dataframe.head()
weather_dataframe

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,harer,30,ET,1650739347,63,9.3094,42.1258,291.15,2.43
1,1,kaitangata,100,NZ,1650739091,91,-46.2817,169.8464,283.64,4.93
2,2,ushuaia,75,AR,1650739348,71,-54.8000,-68.3000,280.96,4.12
3,3,sorland,86,NO,1650739348,83,67.6670,12.6934,278.42,3.74
4,4,olot,100,ES,1650739348,62,42.1810,2.4901,283.25,4.97
...,...,...,...,...,...,...,...,...,...,...
545,545,kenai,75,US,1650739206,63,60.5544,-151.2583,282.61,3.60
546,546,oranjemund,4,NaN,1650739506,85,-28.5500,16.4333,286.50,7.46
547,547,maceio,75,BR,1650739309,61,-9.6658,-35.7353,302.84,4.63
548,548,dahod,0,IN,1650739506,19,22.8333,74.2667,303.97,3.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_dataframe[["Lat","Lng"]].astype(float)

# Convert Humidity Rate to float and store
humidity = weather_dataframe["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities to fit weather conditions.
narrow_dataframe = weather_dataframe.loc[(weather_dataframe["Wind Speed"] <= 10) | (weather_dataframe["Cloudiness"] == 0)| 
                                   (weather_dataframe["Max Temp"] >= 70) | (weather_dataframe["Max Temp"] <= 80)]

narrow_dataframe.dropna()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,harer,30,ET,1650739347,63,9.3094,42.1258,291.15,2.43
1,1,kaitangata,100,NZ,1650739091,91,-46.2817,169.8464,283.64,4.93
2,2,ushuaia,75,AR,1650739348,71,-54.8000,-68.3000,280.96,4.12
3,3,sorland,86,NO,1650739348,83,67.6670,12.6934,278.42,3.74
4,4,olot,100,ES,1650739348,62,42.1810,2.4901,283.25,4.97
...,...,...,...,...,...,...,...,...,...,...
544,544,ayan,59,RU,1650739505,91,56.4500,138.1667,266.69,1.68
545,545,kenai,75,US,1650739206,63,60.5544,-151.2583,282.61,3.60
547,547,maceio,75,BR,1650739309,61,-9.6658,-35.7353,302.84,4.63
548,548,dahod,0,IN,1650739506,19,22.8333,74.2667,303.97,3.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrow_dataframe.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,harer,ET,9.3094,42.1258,
1,kaitangata,NZ,-46.2817,169.8464,
2,ushuaia,AR,-54.8000,-68.3000,
3,sorland,NO,67.6670,12.6934,
4,olot,ES,42.1810,2.4901,
...,...,...,...,...,...
545,kenai,US,60.5544,-151.2583,
546,oranjemund,NaN,-28.5500,16.4333,
547,maceio,BR,-9.6658,-35.7353,
548,dahod,IN,22.8333,74.2667,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)


Retrieving Results for Index 0: harer.
Closest hotel in harer is Sumeya Hotel.
------------
Retrieving Results for Index 1: kaitangata.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: ushuaia.
Closest hotel in ushuaia is Arakur Ushuaia Resort & Spa.
------------
Retrieving Results for Index 3: sorland.
Closest hotel in sorland is Lofoten Værøy Brygge.
------------
Retrieving Results for Index 4: olot.
Closest hotel in olot is Hotel Mas la Ferreria.
------------
Retrieving Results for Index 5: krasnovishersk.
Closest hotel in krasnovishersk is Vishera hotel.
------------
Retrieving Results for Index 6: mataura.
Closest hotel in mataura is Falls Hotel.
------------
Retrieving Results for Index 7: vaini.
Closest hotel in vaini is Paradise First Hotel.
------------
Retrieving Results for Index 8: rikitea.
Closest hotel in rikitea is People ThankYou.
------------
Retrieving Results for Index 9: faya.
Missing field/result... skipping.
------------
Retrieving Re

Missing field/result... skipping.
------------
Retrieving Results for Index 79: ponta do sol.
Closest hotel in ponta do sol is Enotel Sunset Bay.
------------
Retrieving Results for Index 80: sobolevo.
Missing field/result... skipping.
------------
Retrieving Results for Index 81: bredasdorp.
Closest hotel in bredasdorp is Victoria Hotel - Bredasdorp.
------------
Retrieving Results for Index 82: axim.
Closest hotel in axim is Axim Beach Resort & Spa.
------------
Retrieving Results for Index 83: bisert.
Missing field/result... skipping.
------------
Retrieving Results for Index 84: aklavik.
Closest hotel in aklavik is Sundog Inn.
------------
Retrieving Results for Index 85: lakes entrance.
Closest hotel in lakes entrance is The Esplanade Resort & Spa.
------------
Retrieving Results for Index 86: mount gambier.
Closest hotel in mount gambier is Mount Gambier Hotel.
------------
Retrieving Results for Index 87: mayumba.
Closest hotel in mayumba is LIKWALE LODGE.
------------
Retrievin

Closest hotel in port keats is TDC Village (Hundred Man Camp).
------------
Retrieving Results for Index 158: ust-maya.
Missing field/result... skipping.
------------
Retrieving Results for Index 159: leningradskiy.
Closest hotel in leningradskiy is Gostinitsa.
------------
Retrieving Results for Index 160: kaeo.
Closest hotel in kaeo is Pacific Harbour Lodge.
------------
Retrieving Results for Index 161: centralia.
Closest hotel in centralia is Centralia Square Grand Ballroom and Hotel.
------------
Retrieving Results for Index 162: mizoch.
Missing field/result... skipping.
------------
Retrieving Results for Index 163: pangnirtung.
Closest hotel in pangnirtung is Pangnirtung Fjordview Bed and Breakfast.
------------
Retrieving Results for Index 164: mar del plata.
Closest hotel in mar del plata is NH Gran Hotel Provincial.
------------
Retrieving Results for Index 165: cherskiy.
Missing field/result... skipping.
------------
Retrieving Results for Index 166: anadyr.
Closest hotel in

Missing field/result... skipping.
------------
Retrieving Results for Index 236: fort morgan.
Closest hotel in fort morgan is Hampton Inn Fort Morgan.
------------
Retrieving Results for Index 237: guaymas.
Closest hotel in guaymas is Hotel Playa de Cortes.
------------
Retrieving Results for Index 238: tautira.
Missing field/result... skipping.
------------
Retrieving Results for Index 239: nouadhibou.
Closest hotel in nouadhibou is HOtel Tasiast.
------------
Retrieving Results for Index 240: wanning.
Closest hotel in wanning is Victoria Hotel.
------------
Retrieving Results for Index 241: huntsville.
Closest hotel in huntsville is Embassy Suites by Hilton Huntsville Hotel & Spa.
------------
Retrieving Results for Index 242: atar.
Closest hotel in atar is les toiles maures.
------------
Retrieving Results for Index 243: sao gabriel da cachoeira.
Closest hotel in sao gabriel da cachoeira is HOTEL DEUS ME DEU.
------------
Retrieving Results for Index 244: bathsheba.
Closest hotel in

Closest hotel in loandjili is Atlantic Palace Hotel.
------------
Retrieving Results for Index 313: beliu.
Missing field/result... skipping.
------------
Retrieving Results for Index 314: gandai.
Closest hotel in gandai is Ganpati Hotel.
------------
Retrieving Results for Index 315: baramati.
Closest hotel in baramati is Hotel City Inn.
------------
Retrieving Results for Index 316: trelew.
Closest hotel in trelew is Hotel Libertador.
------------
Retrieving Results for Index 317: sehnde.
Closest hotel in sehnde is Hotel Lane.
------------
Retrieving Results for Index 318: varkkallai.
Closest hotel in varkkallai is InDa Hotel.
------------
Retrieving Results for Index 319: lagoa.
Closest hotel in lagoa is Graciosa Resort - Biosphere Island Hotel.
------------
Retrieving Results for Index 320: honningsvag.
Closest hotel in honningsvag is The View.
------------
Retrieving Results for Index 321: iqaluit.
Closest hotel in iqaluit is Aqsarniit Hotel & Conference Centre.
------------
Retrie

Closest hotel in port-gentil is Hotel Mandji.
------------
Retrieving Results for Index 389: mamallapuram.
Closest hotel in mamallapuram is Hotel Mahabs.
------------
Retrieving Results for Index 390: ghorawal.
Closest hotel in ghorawal is सिरसाई.
------------
Retrieving Results for Index 391: la baule-escoublac.
Closest hotel in la baule-escoublac is Hôtel Ty Gwenn.
------------
Retrieving Results for Index 392: bystryy istok.
Closest hotel in bystryy istok is Bumbarash.
------------
Retrieving Results for Index 393: motygino.
Closest hotel in motygino is Gostinitsa Ip Zabirov.
------------
Retrieving Results for Index 394: muroto.
Closest hotel in muroto is Ake no Hoshi.
------------
Retrieving Results for Index 395: uyuni.
Closest hotel in uyuni is Las Tholas Hotel.
------------
Retrieving Results for Index 396: praia.
Closest hotel in praia is Hotel Oasis Atlantico Praiamar.
------------
Retrieving Results for Index 397: voh.
Closest hotel in voh is Le gîte du Koniambo.
-----------

Closest hotel in pala is Hotel Pala Palace.
------------
Retrieving Results for Index 468: conceicao do araguaia.
Closest hotel in conceicao do araguaia is JAMBOS HOTEL.
------------
Retrieving Results for Index 469: santa isabel.
Closest hotel in santa isabel is Hotel Estância Santa Isabel.
------------
Retrieving Results for Index 470: mercedes.
Closest hotel in mercedes is Gran Hotel Mercedes.
------------
Retrieving Results for Index 471: visby.
Closest hotel in visby is Hotell Slottsbacken.
------------
Retrieving Results for Index 472: kantunilkin.
Closest hotel in kantunilkin is Villas Kintunilkan (Restaurante - Hotel).
------------
Retrieving Results for Index 473: nelson bay.
Closest hotel in nelson bay is Hotel Nelson.
------------
Retrieving Results for Index 474: broken hill.
Closest hotel in broken hill is ibis Styles Broken Hill.
------------
Retrieving Results for Index 475: fort smith.
Closest hotel in fort smith is Wyndham Fort Smith City Center.
------------
Retrievin

Closest hotel in kenai is Kenai Airport Hotel.
------------
Retrieving Results for Index 546: oranjemund.
Closest hotel in oranjemund is Oryx Guesthouse.
------------
Retrieving Results for Index 547: maceio.
Closest hotel in maceio is Maceió Atlantic Suites.
------------
Retrieving Results for Index 548: dahod.
Closest hotel in dahod is AVANTIKA RESORT.
------------
Retrieving Results for Index 549: le vauclin.
Closest hotel in le vauclin is Hôtel Cap Macabou.
------------


In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,harer,ET,9.3094,42.1258,Sumeya Hotel
1,kaitangata,NZ,-46.2817,169.8464,
2,ushuaia,AR,-54.8000,-68.3000,Arakur Ushuaia Resort & Spa
3,sorland,NO,67.6670,12.6934,Lofoten Værøy Brygge
4,olot,ES,42.1810,2.4901,Hotel Mas la Ferreria
...,...,...,...,...,...
545,kenai,US,60.5544,-151.2583,Kenai Airport Hotel
546,oranjemund,NaN,-28.5500,16.4333,Oryx Guesthouse
547,maceio,BR,-9.6658,-35.7353,Maceió Atlantic Suites
548,dahod,IN,22.8333,74.2667,AVANTIKA RESORT


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))